***
### Ansible inventory (hosts):

In [ ]:
%%file ./hosts
localhost              ansible_connection=local

v112 ansible_host=192.168.0.112
ubuka ansible_host=192.168.0.236
u100 ansible_host=192.168.0.100 
u101 ansible_host=192.168.0.101 
u102 ansible_host=192.168.0.102 
u103 ansible_host=192.168.0.103 
u104 ansible_host=192.168.0.104 
u105 ansible_host=192.168.0.105 
u106 ansible_host=192.168.0.106 
u107 ansible_host=192.168.0.107 
u108 ansible_host=192.168.0.108 
u109 ansible_host=192.168.0.109 
u110 ansible_host=192.168.0.110 
u111 ansible_host=192.168.0.111 
u112 ansible_host=192.168.0.112 
u113 ansible_host=192.168.0.113
u114 ansible_host=192.168.0.114 
u115 ansible_host=192.168.0.115 
u116 ansible_host=192.168.0.116 
u117 ansible_host=192.168.0.117 
u118 ansible_host=192.168.0.118 
u119 ansible_host=192.168.0.119 
u120 ansible_host=192.168.0.120 
u121 ansible_host=192.168.0.121 
u122 ansible_host=192.168.0.122 
u123 ansible_host=192.168.0.123 
u124 ansible_host=192.168.0.124 
u125 ansible_host=192.168.0.125 
u126 ansible_host=192.168.0.126 
u127 ansible_host=192.168.0.127 
u128 ansible_host=192.168.0.128 
u129 ansible_host=192.168.0.129 
u130 ansible_host=192.168.0.130 

amazon1 ansible_host=52.59.217.167

docker1 ansible_host=www.unibox.hu

[vbox]
v112
ubuka
[vbox:vars]
ansible_port=22 
ansible_connection=ssh 
ansible_python_interpreter=/usr/bin/python3
ansible_user=ubuntu


[tim]
u[101:130]

[tim:vars]
ansible_port=22 
ansible_connection=ssh 
ansible_python_interpreter=/usr/bin/python3
ansible_user=sis


[ubuntu]
server1

[ubuntu:vars]
ansible_port=22
ansible_connection=ssh
ansible_python_interpreter=/usr/bin/python3

[amazon]
amazon1

[amazon:vars]
ansible_port=22
ansible_connection=ssh
ansible_user=ubuntu
ansible_ssh_private_key_file=/home/sis/.ssh/amazon1.pem
ansible_python_interpreter=/usr/bin/python3



***
### Jelszó nélküli sudo a localhoston:

In [ ]:
%%file ./sudo-group.j2
# {{ ansible_managed }}
%sudo ALL=(ALL) NOPASSWD:ALL

In [ ]:
%%sh
xterm -e "sudo cp ./sudo-group.j2 /etc/sudoers.d/95-sudo-group &&sudo chmod 0440 /etc/sudoers.d/95-sudo-group"

***
### GRUB beállítása
sudo add-apt-repository ppa:danielrichter2007/grub-customizer && sudo apt-get update && sudo apt-get install grub-customizer

***
### VirtualBox telepítés:

In [ ]:
%%file ./virtualbox-install.sh
#!/bin/sh
sudo apt-get update && sudo apt-get dist-upgrade && sudo apt-get autoremove
sudo apt-get -y install gcc make linux-headers-$(uname -r) dkms
wget -q https://www.virtualbox.org/download/oracle_vbox_2016.asc -O- | sudo apt-key add -
wget -q https://www.virtualbox.org/download/oracle_vbox.asc -O- | sudo apt-key add -
sudo sh -c 'echo "deb http://download.virtualbox.org/virtualbox/debian $(lsb_release -sc) contrib" >> /etc/apt/sources.list'
sudo apt-get update
sudo apt-get install virtualbox-5.2
VBoxManage -v
 https://download.virtualbox.org/virtualbox/5.2.8/

curl -O http://download.virtualbox.org/virtualbox/5.2.8/Oracle_VM_VirtualBox_Extension_Pack-5.2.8.vbox-extpack
sudo VBoxManage extpack install Oracle_VM_VirtualBox_Extension_Pack-5.2.8.vbox-extpack
VBoxManage list extpacks


In [ ]:
%%sh
chmod +x ./virtualbox-install.sh
xterm -e "./virtualbox-install.sh"
rm ./virtualbox-install.sh

***
### Felügyelet nélküli ubuntu 16.04 telepítő (iso) készítése:

In [ ]:
%%sh
cd ~
git clone https://github.com/englert/ubuntu-iso.git
cd ~/ubuntu-iso
gnome-terminal -e "sudo ./create-unattended-iso.sh"


***
### Ubuntu szerver telepítése Virtualboxon:

In [2]:
%%sh
VM="daniel"
RAM="2048"
VRAM="16"
HDD="8192"
LAN="enp3s0"
ISO="ubuntuserver.iso"
VMs="VirtualBox VMs"
cd ~
VBoxManage createvm --name $VM --ostype "Linux_64" --register
VBoxManage modifyvm $VM --nic1 bridged --bridgeadapter1 $LAN
VBoxManage createhd --filename "$VMs/$VM/$VM.vdi" --size $HDD
VBoxManage storagectl $VM --name "SATA Controller" --add sata
VBoxManage storageattach $VM --storagectl "SATA Controller" --port 0 --device 0 --type hdd --medium "$VMs/$VM/$VM.vdi"
VBoxManage storagectl $VM --name "IDE Controller" --add ide
VBoxManage storageattach $VM --storagectl "IDE Controller" --port 0 --device 0 --type dvddrive --medium $ISO
VBoxManage modifyvm $VM --ioapic on
VBoxManage modifyvm $VM --boot1 dvd --boot2 disk --boot3 none --boot4 none
VBoxManage modifyvm $VM --memory $RAM --vram $VRAM

Virtual machine 'daniel' is created and registered.
UUID: c696fce6-1291-426a-a8bc-eed8e85b64d7
Settings file: '/home/sis/VirtualBox VMs/daniel/daniel.vbox'
Medium created. UUID: be68dbc7-1425-4940-b9b7-501a82bd1e46


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


In [4]:
%%sh
VM="daniel"
VBoxManage startvm $VM --type headless

Waiting for VM "daniel" to power on...
VM "daniel" has been successfully started.


In [ ]:
!ansible --version

***
### ssh-keys feltöltése:

In [ ]:
%%file ./upload_ssh-keys.yml
---
- hosts: ubuka
  remote_user: ubuntu
  tasks:
  - name: upload ssh key                
    authorized_key:
      user: ubuntu
      state: present
      manage_dir: yes
      key: https://github.com/englert.keys


In [ ]:
%%bash
xterm -e "ansible-playbook -i ./hosts ./upload_ssh-keys.yml -k"
rm ./upload_ssh-keys.yml

***
### Host ellenőrzése: ansible ping 

In [ ]:
!ansible -i ./hosts -m ping  ubuka

***
### Ansible jelszó nélküli sudo 

In [ ]:
%%file ./sudo-group.j2
# {{ ansible_managed }}
%sudo ALL=(ALL) NOPASSWD:ALL

In [ ]:
%%file ./passwordless-sudo.yml
---
- hosts: ubuka
  remote_user: ubuntu
  become: true
  tasks:

  - name:  Legyen sudo csoport
    group:
      name: sudo
      state: present

  - name: A sudo csoport felhasználói jelszó nélkül használhatják a sudo-t.
    template:
      src: sudo-group.j2
      dest: /etc/sudoers.d/95-sudo-group
      mode: 0440

In [ ]:
%%bash
xterm -e "ansible-playbook -i ./hosts -l ubuka ./passwordless-sudo.yml -K"
rm ./passwordless-sudo.yml

***
### Program telepítése Ansible ad-hoc módban
ansible -i ./hosts --become -m apt -a"name=mc state=present" amazon1

In [ ]:
!ansible -i ./hosts --become -m apt -a"name=mc state=present" ubuka

***
### Nginx telepítés ansible-playbookkal: 

In [ ]:
%%file ./index.html
<html>
  <H1> Nginx szerver </H1>
</html>

In [ ]:
%%file ./nginx.yml
---
- name: Install nginx
  hosts: amazon1
  become: true

  tasks:
  
  - name: Install nginx
    apt:
      name: nginx
      state: present

  - name: Insert Index Page
    template:
      src: index.html
      dest: /usr/share/nginx/html/index.html

  - name: Start NGiNX
    service:
      name: nginx
      state: started


In [ ]:
!ansible-playbook -i ./hosts ./nginx.yml -l amazon

***
### docker telepítése:

In [ ]:
!ansible-galaxy install nickjj.docker


In [ ]:
!mkdir group_vars

In [ ]:
%%file site.yml
---

- name: "Configure app server(s)"
  hosts: "ubuka"
  become: True

  roles:
    - { role: "nickjj.docker", tags: "docker" }

In [ ]:
%%file group_vars/app.yml
---

docker_users: ["ubuntu"]

In [ ]:
!ansible-playbook -i ./hosts site.yml -t docker 

***
### Docker konténerek automatikus indítása

A docker container-ek a host OS újraindítása után sajnos nem indulnak el maguktól.
Ezt az alábbi egyszerű scripttel oldotta meg SAndras .
https://medium.com/sandras-techblog/docker-container-technol%C3%B3gia-f94f843cac6c

In [ ]:
%%bash
sudo echo "/opt/docker-startcontainer.sh" >>/etc/rc.local

In [ ]:
%%file docker-startcontainer.sh
#!/bin/bash
# A script a "docker ps -a"-val lekérdezi az elérhető containerek listáját és mindegyiket elindítja.
# start all stopped containers
for id in `docker ps -a | cut -f1 -d' ' | grep -iv container`
# start containers that ran/exited in the last some minutes os seconds
#for id in `docker ps -a | egrep -i exited.*'(min|sec)'.*ago | cut -f1 -d' '`
do
  docker start $id
  # minimal timeout after start of the container
  sleep 5
  # run the service start script in the container
  #docker exec $id /bin/bash /opt/start.sh
done

In [ ]:
%%bash
chmod +x docker-startcontainer.sh
sudo cp docker-startcontainer.sh /opt/docker-startcontainer.sh


***
### Ansible user 

In [ ]:
!ansible-galaxy install nickjj.user
!mkdir inventory
!cp hosts inventory/hosts

In [ ]:
%%file site.yml
---

- name: "Configure app server(s)"
  hosts: "v112"
  become: True

  roles:
    - { role: "nickjj.user", tags: "user" }

In [ ]:
%%file inventory/app.yml
---

user_name: "thor"

In [ ]:
!ansible-playbook -i inventory/hosts site.yml -t user